# All Imports

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, timedelta


# File loading

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
df_upload = pd.read_csv(io.BytesIO(uploaded['factTrans.csv']))
# Dataset is now stored in a Pandas Dataframe

Saving factTrans.csv to factTrans.csv


# EDA

In [ ]:
df_upload.head()
print(df_upload['BRANCH_ID'].unique())
print(df_upload['TYPE_OF_TRANSACTION'].unique())


[1 2 3 4]
['INFLOW' 'OUTFLOW']


In [ ]:
df = df_upload
# print(df)
df.describe()
df.count()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6565 entries, 0 to 6564
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              6565 non-null   int64  
 1   BRANCH_ID                       6565 non-null   int64  
 2   TRANS_DATE                      6565 non-null   object 
 3   TYPE_OF_TRANSACTION             6565 non-null   object 
 4   DATEWISE_INFLOW_OUTFLOW_AMOUNT  6565 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 256.6+ KB


,ID,BRANCH_ID,TRANS_DATE,TYPE_OF_TRANSACTION,DATEWISE_INFLOW_OUTFLOW_AMOUNT
0,1,1,8/2/2018,INFLOW,114000.00
1,2,1,8/2/2018,OUTFLOW,206152.00
2,3,1,8/5/2018,INFLOW,121300.00
3,4,1,8/5/2018,OUTFLOW,83300.00
4,5,1,8/6/2018,INFLOW,20000.34


In [ ]:
df=df.drop('ID', axis=1)
df.head()

,BRANCH_ID,TRANS_DATE,TYPE_OF_TRANSACTION,DATEWISE_INFLOW_OUTFLOW_AMOUNT
0,1,8/2/2018,INFLOW,114000.00
1,1,8/2/2018,OUTFLOW,206152.00
2,1,8/5/2018,INFLOW,121300.00
3,1,8/5/2018,OUTFLOW,83300.00
4,1,8/6/2018,INFLOW,20000.34


In [ ]:
def transform_date(date):
    flags = date.split('/')
    date = datetime.date(int(flags[2]), int(flags[0]), int(flags[1]))
    return date

df_date = df.TRANS_DATE.apply(transform_date)
df.Date = df_date
df

<ipython-input-6-45028ffca965>:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Date = df_date


,BRANCH_ID,TRANS_DATE,TYPE_OF_TRANSACTION,DATEWISE_INFLOW_OUTFLOW_AMOUNT
0,1,8/2/2018,INFLOW,114000.00
1,1,8/2/2018,OUTFLOW,206152.00
2,1,8/5/2018,INFLOW,121300.00
3,1,8/5/2018,OUTFLOW,83300.00
4,1,8/6/2018,INFLOW,20000.34
...,...,...,...,...
6560,4,7/18/2022,INFLOW,1000.00
6561,4,7/18/2022,OUTFLOW,513942.00
6562,4,7/19/2022,OUTFLOW,117641.00
6563,4,7/20/2022,INFLOW,2500.00


# Inflow Outflow columns rearranging

In [ ]:
def daterange(start_date, end_date):
        for n in range(int((end_date - start_date).days)):
            yield start_date + timedelta(n)

In [ ]:
cols = ['BRANCH_ID', 'TRANS_DATE', 'INFLOW', 'OUTFLOW']
df_in_out = pd.DataFrame(columns=cols)

In [ ]:
# start_date = df.Date.min()
# end_date = df.Date.max()

# for branch in (df_upload['BRANCH_ID'].unique()):
#     df2 = df[df['BRANCH_ID']==branch]
#     for single_date in daterange(start_date, end_date):
#         x = str(single_date)
#         date = str(int(x[-2:]))+'/'+ str(int(x[-5:-3]))+'/'+x[:4]
#         try:
#             df3 = df2[df2['TRANS_DATE']==date]
#             df4 = df3[df3['TYPE_OF_TRANSACTION']=='INFLOW']
#             df5 = df3[df3['TYPE_OF_TRANSACTION']=='OUTFLOW']
#             df4 = df4.reset_index()
#             df5 = df5.reset_index()
#             # print(df4['DATEWISE_INFLOW_OUTFLOW_AMOUNT'][0])
#             df_tmp = pd.DataFrame({'BRANCH_ID': [branch], 'TRANS_DATE': [date], 'INFLOW': [df4['DATEWISE_INFLOW_OUTFLOW_AMOUNT'][0]], 'OUTFLOW': [df5['DATEWISE_INFLOW_OUTFLOW_AMOUNT'][0]]})
#         except:
#             df_tmp = pd.DataFrame({'BRANCH_ID': [branch], 'TRANS_DATE': [date], 'INFLOW': [0], 'OUTFLOW': [0]})
#         df_in_out = pd.concat([df_in_out, df_tmp], ignore_index = True)

In [ ]:
start_date = df.Date.min()
end_date = df.Date.max()

for branch in (df_upload['BRANCH_ID'].unique()):
    df2 = df[df['BRANCH_ID']==branch]
    for single_date in daterange(start_date, end_date):
        df3 = df2[df2['TRANS_DATE']==single_date]
        df_tmp = pd.DataFrame({
            'BRANCH_ID': [branch], 
            'TRANS_DATE': [single_date], 
            'INFLOW': [0], 
            'OUTFLOW': [0]
        })
        for idx in df3.index:
            df_tmp[df3.loc[idx, 'TYPE_OF_TRANSACTION']] = df3.loc[idx, 'DATEWISE_INFLOW_OUTFLOW_AMOUNT']
        df_in_out = pd.concat([df_in_out, df_tmp], ignore_index = True)

In [ ]:
df_in_out.to_csv('transDataInOut.csv', encoding = 'utf-8-sig') 
files.download('transDataInOut.csv')
df_in_out.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,BRANCH_ID,TRANS_DATE,INFLOW,OUTFLOW
0,1,2018-08-02,0,0
1,1,2018-08-03,0,0
2,1,2018-08-04,0,0
3,1,2018-08-05,0,0
4,1,2018-08-06,0,0


In [ ]:
df_in_out.BRANCH_ID.unique()

array([1, 2, 3, 4], dtype=object)

# Preparing New Data

In [ ]:
uploaded = files.upload()
df_upload = pd.read_csv(io.BytesIO(uploaded['transDataInOut.csv']))
df = df_upload

Saving transDataInOut.csv to transDataInOut (2).csv


## OR

In [ ]:
df = df_in_out

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5792 entries, 0 to 5791
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BRANCH_ID   5792 non-null   object
 1   TRANS_DATE  5792 non-null   object
 2   INFLOW      5792 non-null   object
 3   OUTFLOW     5792 non-null   object
dtypes: object(4)
memory usage: 181.1+ KB


In [ ]:
df.head()

,BRANCH_ID,TRANS_DATE,INFLOW,OUTFLOW
0,1,2018-08-02,0,0
1,1,2018-08-03,0,0
2,1,2018-08-04,0,0
3,1,2018-08-05,0,0
4,1,2018-08-06,0,0


In [ ]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])

In [ ]:
df = df[['BRANCH_ID','TRANS_DATE','INFLOW', 'OUTFLOW']]
df=df.rename(columns = {'TRANS_DATE':'Date'})
df.head()

,BRANCH_ID,Date,INFLOW,OUTFLOW
0,1,2018-08-02,0,0
1,1,2018-08-03,0,0
2,1,2018-08-04,0,0
3,1,2018-08-05,0,0
4,1,2018-08-06,0,0


In [ ]:
df.set_index('Date', inplace=True)
df


,BRANCH_ID,INFLOW,OUTFLOW
Date,,,
2018-08-02,1,0,0
2018-08-03,1,0,0
2018-08-04,1,0,0
2018-08-05,1,0,0
2018-08-06,1,0,0
...,...,...,...
2022-07-15,4,0,0
2022-07-16,4,0,0
2022-07-17,4,0,0


# DF segmentation


In [ ]:
df_1 = df[(df['BRANCH_ID'] == 1)]
df_2 = df[(df['BRANCH_ID'] == 2)]
df_3 = df[(df['BRANCH_ID'] == 3)]
df_4 = df[(df['BRANCH_ID'] == 4)]
df_1.info()

all_dfs = [df_1, df_2, df_3, df_4]
all_dfs

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1448 entries, 2018-08-02 to 2022-07-19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   BRANCH_ID  1448 non-null   object
 1   INFLOW     1448 non-null   object
 2   OUTFLOW    1448 non-null   object
dtypes: object(3)
memory usage: 45.2+ KB


[           BRANCH_ID INFLOW OUTFLOW
 Date                               
 2018-08-02         1      0       0
 2018-08-03         1      0       0
 2018-08-04         1      0       0
 2018-08-05         1      0       0
 2018-08-06         1      0       0
 ...              ...    ...     ...
 2022-07-15         1      0       0
 2022-07-16         1      0       0
 2022-07-17         1      0       0
 2022-07-18         1      0       0
 2022-07-19         1      0       0
 
 [1448 rows x 3 columns],
            BRANCH_ID INFLOW OUTFLOW
 Date                               
 2018-08-02         2      0       0
 2018-08-03         2      0       0
 2018-08-04         2      0       0
 2018-08-05         2      0       0
 2018-08-06         2      0       0
 ...              ...    ...     ...
 2022-07-15         2      0       0
 2022-07-16         2      0       0
 2022-07-17         2      0       0
 2022-07-18         2      0       0
 2022-07-19         2      0       0
 
 [1448 r

# Adding Days
Days are added for all data frames for the entire initial df

In [ ]:
def get_alldates(start_date, end_date):
      df_weekdays = pd.DataFrame()
      days = ['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']
      delta = datetime.timedelta(days=1)
      dates = []
      week_day = []
      cur = 6
      while start_date<=end_date:
          dates.append(start_date)
          week_day.append(days[cur])
          start_date += delta
          cur = (cur+1)%7
      df_weekdays = pd.DataFrame(data=dates, columns=['Date'])
      df_weekdays['weekdays'] = week_day
      return df_weekdays,dates

# all_dfs = [df_1_out, df_1_in, df_2_out, df_2_in, df_3_out, df_3_in, df_4_out, df_4_in]


In [ ]:
df_final_weekdays = pd.DataFrame()
df_final_list = []
for single_df in all_dfs:
    df_weekdays, dates=get_alldates(single_df.index.min(), single_df.index.max())
    outflow = []
    inflow = []
    BRANCH_ID = []

    for date in dates:
        BRANCH_ID.append(single_df['BRANCH_ID'].iat[0])
        try:
            outflow.append(single_df.loc[date]['OUTFLOW'])
        except:
            outflow.append(0)
        try:
            inflow.append(single_df.loc[date]['INFLOW'])
        except:
            inflow.append(0)
    df_weekdays['OUTFLOW'] = outflow
    df_weekdays['INFLOW'] = inflow
    df_weekdays['BRANCH_ID'] = BRANCH_ID
    print('Each dfs ',df_weekdays)
    frames = [df_final_weekdays, df_weekdays]
    df_final_weekdays = pd.concat(frames)
df_final_weekdays

Each dfs             Date  weekdays  OUTFLOW  INFLOW  BRANCH_ID
0    2018-08-02  Thursday        0       0          1
1    2018-08-03    Friday        0       0          1
2    2018-08-04  Saturday        0       0          1
3    2018-08-05    Sunday        0       0          1
4    2018-08-06    Monday        0       0          1
...         ...       ...      ...     ...        ...
1443 2022-07-15    Friday        0       0          1
1444 2022-07-16  Saturday        0       0          1
1445 2022-07-17    Sunday        0       0          1
1446 2022-07-18    Monday        0       0          1
1447 2022-07-19   Tuesday        0       0          1

[1448 rows x 5 columns]
Each dfs             Date  weekdays  OUTFLOW  INFLOW  BRANCH_ID
0    2018-08-02  Thursday        0       0          2
1    2018-08-03    Friday        0       0          2
2    2018-08-04  Saturday        0       0          2
3    2018-08-05    Sunday        0       0          2
4    2018-08-06    Monday        0   

,Date,weekdays,OUTFLOW,INFLOW,BRANCH_ID
0,2018-08-02,Thursday,0,0,1
1,2018-08-03,Friday,0,0,1
2,2018-08-04,Saturday,0,0,1
3,2018-08-05,Sunday,0,0,1
4,2018-08-06,Monday,0,0,1
...,...,...,...,...,...
1443,2022-07-15,Friday,0,0,4
1444,2022-07-16,Saturday,0,0,4
1445,2022-07-17,Sunday,0,0,4
1446,2022-07-18,Monday,0,0,4


# Download final DF as csv file

In [ ]:
df_final_weekdays.to_csv('transDataDates.csv', encoding = 'utf-8-sig') 
files.download('transDataDates.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# File loading new IN OUT with Dates File

In [ ]:
uploaded = files.upload()
df_upload = pd.read_csv(io.BytesIO(uploaded['transDataDates.csv']))
df = df_upload

Saving transDataDates.csv to transDataDates (2).csv


## OR

In [ ]:
df = df_final_weekdays
df.head()
print(len(df))
df.BRANCH_ID.unique()

5792


array([1, 2, 3, 4])

# EDA

In [ ]:
print(len(df))
df.describe()
df.count()
df.info()
df.head()

5792
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5792 entries, 0 to 1447
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       5792 non-null   datetime64[ns]
 1   weekdays   5792 non-null   object        
 2   OUTFLOW    5792 non-null   int64         
 3   INFLOW     5792 non-null   int64         
 4   BRANCH_ID  5792 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 271.5+ KB


,Date,weekdays,OUTFLOW,INFLOW,BRANCH_ID
0,2018-08-02,Thursday,0,0,1
1,2018-08-03,Friday,0,0,1
2,2018-08-04,Saturday,0,0,1
3,2018-08-05,Sunday,0,0,1
4,2018-08-06,Monday,0,0,1


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()
df = df[['weekdays', 'OUTFLOW','INFLOW', 'BRANCH_ID']]
df

,weekdays,OUTFLOW,INFLOW,BRANCH_ID
Date,,,,
2018-08-02,Thursday,0,0,1
2018-08-03,Friday,0,0,1
2018-08-04,Saturday,0,0,1
2018-08-05,Sunday,0,0,1
2018-08-06,Monday,0,0,1
...,...,...,...,...
2022-07-15,Friday,0,0,4
2022-07-16,Saturday,0,0,4
2022-07-17,Sunday,0,0,4


# Data Segmentation

In [ ]:
df_1 = df[(df['BRANCH_ID'] == 1)]
df_2 = df[(df['BRANCH_ID'] == 2)]
df_3 = df[(df['BRANCH_ID'] == 3)]
df_4 = df[(df['BRANCH_ID'] == 4)]
df_1.info()

all_dfs = [df_1, df_2, df_3, df_4]
all_dfs

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1448 entries, 2018-08-02 to 2022-07-19
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   weekdays   1448 non-null   object
 1   OUTFLOW    1448 non-null   int64 
 2   INFLOW     1448 non-null   int64 
 3   BRANCH_ID  1448 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 56.6+ KB


[            weekdays  OUTFLOW  INFLOW  BRANCH_ID
 Date                                            
 2018-08-02  Thursday        0       0          1
 2018-08-03    Friday        0       0          1
 2018-08-04  Saturday        0       0          1
 2018-08-05    Sunday        0       0          1
 2018-08-06    Monday        0       0          1
 ...              ...      ...     ...        ...
 2022-07-15    Friday        0       0          1
 2022-07-16  Saturday        0       0          1
 2022-07-17    Sunday        0       0          1
 2022-07-18    Monday        0       0          1
 2022-07-19   Tuesday        0       0          1
 
 [1448 rows x 4 columns],
             weekdays  OUTFLOW  INFLOW  BRANCH_ID
 Date                                            
 2018-08-02  Thursday        0       0          2
 2018-08-03    Friday        0       0          2
 2018-08-04  Saturday        0       0          2
 2018-08-05    Sunday        0       0          2
 2018-08-06    Monday 

In [ ]:
# start_date = df.index.min()
# end_date = df.index.max()
# delta = datetime.timedelta(days=1)
# i = start_date
# while i<=end_date:
#     if (df_1.loc[i, 'INFLOW']!=0 and df_1.loc[i, 'OUTFLOW']!=0)\
#         or (df_2.loc[i, 'INFLOW']!=0 and df_2.loc[i, 'OUTFLOW']!=0)\
#         or (df_3.loc[i, 'INFLOW']!=0 and df_3.loc[i, 'OUTFLOW']!=0)\
#         or (df_4.loc[i, 'INFLOW']!=0 and df_4.loc[i, 'OUTFLOW']!=0):
#         df_1 = df_1[i:]
#         df_2 = df_2[i:]
#         df_3 = df_3[i:]
#         df_4 = df_4[i:]
#         break
#     i+=delta

# i = end_date
# while start_date<=i:
#     if (df_1.loc[i, 'INFLOW']!=0 and df_1.loc[i, 'OUTFLOW']!=0)\
#         or (df_2.loc[i, 'INFLOW']!=0 and df_2.loc[i, 'OUTFLOW']!=0)\
#         or (df_3.loc[i, 'INFLOW']!=0 and df_3.loc[i, 'OUTFLOW']!=0)\
#         or (df_4.loc[i, 'INFLOW']!=0 and df_4.loc[i, 'OUTFLOW']!=0):
#         df_1 = df_1[:i]
#         df_2 = df_2[:i]
#         df_3 = df_3[:i]
#         df_4 = df_4[:i]
#         break
#     i-=delta

In [ ]:
len(df_3)

1448

In [ ]:
df_1.head()

,weekdays,OUTFLOW,INFLOW,BRANCH_ID
Date,,,,
2018-08-02,Thursday,0,0,1
2018-08-03,Friday,0,0,1
2018-08-04,Saturday,0,0,1
2018-08-05,Sunday,0,0,1
2018-08-06,Monday,0,0,1


In [ ]:
df.columns

Index(['weekdays', 'OUTFLOW', 'INFLOW', 'BRANCH_ID'], dtype='object')

# Adding Holidays list

In [ ]:
HOLIDAY_LIST = [[2018, 8, 15, 'National Mourning Day'], [2018, 8, 21, 'Eid-ul-Azha'], [2018, 8, 22, 'Eid-ul-Azha'], [2018, 8, 23, 'Eid-ul-Azha'],
                [2018, 9, 2, 'Janmashtami'], [2018, 9, 21, 'Ashura'], [2018, 10, 19, 'Durga Puja'], [2018, 11, 21, 'Eid-e-Miladunnabi'], [2018, 12, 16, 'Victory Day'],
                [2018, 12, 25, 'Christmas Day'], [2018, 12, 30, 'Parlament Election'], [2018, 12, 31, 'Bank Holiday'],
                [2019, 2, 21, 'Language Martyrs\' Day'], [2019, 3, 17, 'Sheikh Mujibur Rahman\'s Birthday'], [2019, 3, 26, 'Independence Day'],
                [2019, 4, 14, 'Bengali New Year'], [2019, 4, 21, 'Shab e-Barat'], [2019, 5, 1, 'Labour Day'], [2019, 5, 18, 'Buddha Purnima'],
                [2019, 5, 31, 'Jumatul Bidah'], [2019, 6, 2, 'Night of Destiny'], [2019, 6, 4, 'Eid al-Fitr'], [2019, 6, 5, 'Eid al-Fitr'], [2019, 6, 6, 'Eid al-Fitr'],
                [2019, 8, 11, 'Eid-ul-Azha'], [2019, 8, 12, 'Eid-ul-Azha'], [2019, 8, 13, 'Eid-ul-Azha'], [2019, 8, 15, 'National Mourning Day'], [2019, 8, 23, 'Janmashtami'],
                [2019, 9, 10, 'Ashura'], [2019, 10, 8, 'Durga Puja'], [2019, 11, 10, 'Eid-e-Miladunnabi'], [2019, 12, 16, 'Victory Day'], [2019, 12, 25, 'Christmas Day'],
                [2020, 2, 21, 'Language Martyrs\' Day'], [2020, 3, 17, 'Sheikh Mujibur Rahman\'s Birthday'], [2020, 3, 26, 'Independence Day'],
                [2020, 4, 14, 'Bengali New Year'], [2020, 4, 9, 'Shab e-Barat'], [2020, 5, 1, 'Labour Day'], [2020, 5, 7, 'Buddha Purnima'],
                [2020, 5, 22, 'Jumatul Bidah'], [2020, 5, 21, 'Night of Destiny'], [2020, 5, 24, 'Eid al-Fitr'], [2020, 5, 25, 'Eid al-Fitr'], [2020, 5, 26, 'Eid al-Fitr'],
                [2020, 7, 31, 'Eid-ul-Azha'], [2020, 8, 1, 'Eid-ul-Azha'], [2020, 8, 11, 'Janmashtami'], [2020, 8, 15, 'National Mourning Day'],
                [2020, 8, 30, 'Ashura'], [2020, 10, 25, 'Durga Puja'], [2020, 10, 30, 'Eid-e-Miladunnabi'], [2020, 12, 16, 'Victory Day'], [2020, 12, 25, 'Christmas Day'],
                [2021, 2, 21, 'Language Martyrs\' Day'], [2021, 3, 17, 'Sheikh Mujibur Rahman\'s Birthday'], [2021, 3, 26, 'Independence Day'],
                [2021, 4, 14, 'Bengali New Year'], [2021, 3, 30, 'Shab e-Barat'], [2021, 5, 1, 'Labour Day'], [2021, 5, 26, 'Buddha Purnima'],
                [2021, 5, 7, 'Jumatul Bidah'], [2021, 5, 10, 'Night of Destiny'], [2021, 5, 14, 'Eid al-Fitr'], [2021, 5, 15, 'Eid al-Fitr'], [2021, 5, 16, 'Eid al-Fitr'],
                [2021, 7, 20, 'Eid-ul-Azha'], [2021, 7, 21, 'Eid-ul-Azha'], [2021, 7, 22, 'Eid-ul-Azha'], [2021, 8, 30, 'Janmashtami'], [2021, 8, 15, 'National Mourning Day'],
                [2021, 8, 19, 'Ashura'], [2021, 10, 15, 'Durga Puja'], [2021, 10, 20, 'Eid-e-Miladunnabi'], [2021, 12, 16, 'Victory Day'], [2021, 12, 25, 'Christmas Day'],
                [2022, 2, 21, 'Language Martyrs\' Day'], [2022, 3, 17, 'Sheikh Mujibur Rahman\'s Birthday'], [2022, 3, 26, 'Independence Day'],
                [2021, 4, 14, 'Bengali New Year'], [2022, 3, 18, 'Shab e-Barat'], [2022, 5, 1, 'Labour Day'], [2022, 5, 16, 'Buddha Purnima'],
                [2022, 4, 29, 'Jumatul Bidah'], [2022, 4, 28, 'Night of Destiny'], [2022, 5, 2, 'Eid al-Fitr'], [2022, 5, 3, 'Eid al-Fitr'], [2022, 5, 4, 'Eid al-Fitr'],
                [2022, 7, 9, 'Eid-ul-Azha'], [2022, 7, 10, 'Eid-ul-Azha'], [2022, 7, 11, 'Eid-ul-Azha']]

In [ ]:
NATIONAL_HOLIDAYS = ['National Mourning Day', 'Victory Day', 'Parlament Election', 'Bank Holiday', 'Language Martyrs\' Day',
                     'Sheikh Mujibur Rahman\'s Birthday', 'Independence Day', 'Labour Day', ]
RELIGIOUS_HOLIDAYS = ['Eid-ul-Azha', 'Janmashtami', 'Ashura', 'Durga Puja', 'Eid-e-Miladunnabi', 'Christmas Day', 'Bengali New Year',
                      'Shab e-Barat', 'Buddha Purnima', 'Jumatul Bidah', 'Night of Destiny', 'Eid al-Fitr']

# Adding holiday types 'NH' (NATIONAL_HOLIDAYS) and 'RH' (RELIGIOUS_HOLIDAYS)

In [ ]:
all_dfs_tmp = []
for df in all_dfs:
    df['is_holiday']=0
    df['holiday_name']=''
    df['holiday_type']=''
    for day in HOLIDAY_LIST:
        date = str(datetime.date(day[0], day[1], day[2]))
        df.loc[date, 'is_holiday'] = 1
        df.loc[date, 'holiday_name'] = day[3]
        if day[3] in NATIONAL_HOLIDAYS:
            df.loc[date, 'holiday_type'] = 'NH'
        elif day[3] in RELIGIOUS_HOLIDAYS:
            df.loc[date, 'holiday_type'] = 'RH'
    all_dfs_tmp.append(df)
all_dfs = all_dfs_tmp
print(all_dfs)

[            weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday holiday_name  \
Date                                                                        
2018-08-02  Thursday        0       0          1           0                
2018-08-03    Friday        0       0          1           0                
2018-08-04  Saturday        0       0          1           0                
2018-08-05    Sunday        0       0          1           0                
2018-08-06    Monday        0       0          1           0                
...              ...      ...     ...        ...         ...          ...   
2022-07-15    Friday        0       0          1           0                
2022-07-16  Saturday        0       0          1           0                
2022-07-17    Sunday        0       0          1           0                
2022-07-18    Monday        0       0          1           0                
2022-07-19   Tuesday        0       0          1           0               

<ipython-input-34-1bfa4c8b479c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_holiday']=0
<ipython-input-34-1bfa4c8b479c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['holiday_name']=''
<ipython-input-34-1bfa4c8b479c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

# Adding holiday type 'WK' (Weekly) to df

In [ ]:

all_dfs = [df_1, df_2, df_3, df_4]
all_dfs_tmp = []

for df in all_dfs:
    start_date = df.index.min()
    end_date = df.index.max()
    delta = datetime.timedelta(days=1)
    while start_date<=end_date:
        if (df.loc[str(start_date), 'weekdays']=='Friday' or df.loc[str(start_date), 'weekdays']=='Saturday') and not df.loc[str(start_date), 'is_holiday']:
            df.loc[str(start_date), 'is_holiday'] = 1
            df.loc[str(start_date), 'holiday_name'] = 'Weekly'
            df.loc[str(start_date), 'holiday_type'] = 'WK'
        start_date += delta
    print(df.head(10))
    all_dfs_tmp.append(df)
all_dfs = all_dfs_tmp
print(all_dfs)

             weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday holiday_name  \
Date                                                                         
2018-08-02   Thursday        0       0          1           0                
2018-08-03     Friday        0       0          1           1       Weekly   
2018-08-04   Saturday        0       0          1           1       Weekly   
2018-08-05     Sunday        0       0          1           0                
2018-08-06     Monday        0       0          1           0                
2018-08-07    Tuesday        0       0          1           0                
2018-08-08  Wednesday        0       0          1           0                
2018-08-09   Thursday        0       0          1           0                
2018-08-10     Friday        0       0          1           1       Weekly   
2018-08-11   Saturday        0       0          1           1       Weekly   

           holiday_type  
Date                     
2018-08-02 

# Adding holiday (3 day) sequence

In [ ]:
for df in all_dfs:
    start_date = df.index.min()
    end_date = df.index.max()
    delta = datetime.timedelta(days=1)

    df.loc[str(end_date), 'holiday_sequence'] = 'WWH'
    end_date-=delta

    while start_date<=end_date:
        # print(df.loc[str(start_date), 'is_holiday'])
        if df.loc[str(end_date), 'is_holiday']==1: ad = 'H'
        else: ad = 'W'
        df.loc[str(end_date), 'holiday_sequence'] = ad+df.loc[str(end_date+delta), 'holiday_sequence'][:-1]
        end_date -= delta
    print(df.head(20))

<ipython-input-36-e2b0c372d5bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[str(end_date), 'holiday_sequence'] = 'WWH'


             weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday  \
Date                                                            
2018-08-02   Thursday        0       0          1           0   
2018-08-03     Friday        0       0          1           1   
2018-08-04   Saturday        0       0          1           1   
2018-08-05     Sunday        0       0          1           0   
2018-08-06     Monday        0       0          1           0   
2018-08-07    Tuesday        0       0          1           0   
2018-08-08  Wednesday        0       0          1           0   
2018-08-09   Thursday        0       0          1           0   
2018-08-10     Friday        0       0          1           1   
2018-08-11   Saturday        0       0          1           1   
2018-08-12     Sunday        0       0          1           0   
2018-08-13     Monday        0       0          1           0   
2018-08-14    Tuesday        0       0          1           0   
2018-08-15  Wednesday    

<ipython-input-36-e2b0c372d5bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[str(end_date), 'holiday_sequence'] = 'WWH'


             weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday  \
Date                                                            
2018-08-02   Thursday        0       0          2           0   
2018-08-03     Friday        0       0          2           1   
2018-08-04   Saturday        0       0          2           1   
2018-08-05     Sunday        0       0          2           0   
2018-08-06     Monday        0       0          2           0   
2018-08-07    Tuesday        0       0          2           0   
2018-08-08  Wednesday        0       0          2           0   
2018-08-09   Thursday        0       0          2           0   
2018-08-10     Friday        0       0          2           1   
2018-08-11   Saturday        0       0          2           1   
2018-08-12     Sunday        0       0          2           0   
2018-08-13     Monday        0       0          2           0   
2018-08-14    Tuesday        0       0          2           0   
2018-08-15  Wednesday    

<ipython-input-36-e2b0c372d5bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[str(end_date), 'holiday_sequence'] = 'WWH'


             weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday  \
Date                                                            
2018-08-02   Thursday        0       0          3           0   
2018-08-03     Friday        0       0          3           1   
2018-08-04   Saturday        0       0          3           1   
2018-08-05     Sunday        0       0          3           0   
2018-08-06     Monday        0       0          3           0   
2018-08-07    Tuesday        0       0          3           0   
2018-08-08  Wednesday        0       0          3           0   
2018-08-09   Thursday        0       0          3           0   
2018-08-10     Friday        0       0          3           1   
2018-08-11   Saturday        0       0          3           1   
2018-08-12     Sunday        0       0          3           0   
2018-08-13     Monday        0       0          3           0   
2018-08-14    Tuesday        0       0          3           0   
2018-08-15  Wednesday    

<ipython-input-36-e2b0c372d5bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[str(end_date), 'holiday_sequence'] = 'WWH'


             weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday  \
Date                                                            
2018-08-02   Thursday        0       0          4           0   
2018-08-03     Friday        0       0          4           1   
2018-08-04   Saturday        0       0          4           1   
2018-08-05     Sunday        0       0          4           0   
2018-08-06     Monday        0       0          4           0   
2018-08-07    Tuesday        0       0          4           0   
2018-08-08  Wednesday        0       0          4           0   
2018-08-09   Thursday        0       0          4           0   
2018-08-10     Friday        0       0          4           1   
2018-08-11   Saturday        0       0          4           1   
2018-08-12     Sunday        0       0          4           0   
2018-08-13     Monday        0       0          4           0   
2018-08-14    Tuesday        0       0          4           0   
2018-08-15  Wednesday    

# Reset all indexes

In [ ]:
all_dfs_tmp = []
for df in all_dfs:
  df = df.reset_index()
  all_dfs_tmp.append(df)
all_dfs = all_dfs_tmp
print(all_dfs)

[           Date  weekdays  OUTFLOW  INFLOW  BRANCH_ID  is_holiday  \
0    2018-08-02  Thursday        0       0          1           0   
1    2018-08-03    Friday        0       0          1           1   
2    2018-08-04  Saturday        0       0          1           1   
3    2018-08-05    Sunday        0       0          1           0   
4    2018-08-06    Monday        0       0          1           0   
...         ...       ...      ...     ...        ...         ...   
1443 2022-07-15    Friday        0       0          1           1   
1444 2022-07-16  Saturday        0       0          1           1   
1445 2022-07-17    Sunday        0       0          1           0   
1446 2022-07-18    Monday        0       0          1           0   
1447 2022-07-19   Tuesday        0       0          1           0   

     holiday_name holiday_type holiday_sequence  
0                                           WHH  
1          Weekly           WK              HHW  
2          Weekly   

# Adding holiday (7, 15, 30 days) sequence

In [ ]:
def checkNextHoliday(df, num):
    for i in range(len(df)):
        for j in range(i+1, min(i+1+num, len(df))):
            if df.loc[j, 'holiday_type']=='RH':
                df.loc[i, 'RH_in_'+str(num)] = 1
                break
    return df

In [ ]:
all_dfs_tmp = []
for df in all_dfs:
    df['RH_in_7'] = 0
    df['RH_in_15'] = 0
    df['RH_in_30'] = 0
    df = checkNextHoliday(df, 7)
    df = checkNextHoliday(df, 15)
    df = checkNextHoliday(df, 30)
    all_dfs_tmp.append(df)

all_dfs = all_dfs_tmp

# Next holiday

In [ ]:

all_dfs_tmp = []
for df in all_dfs:
    df['next_RH'] = 200
    for i in range(len(df)):
        for j in range(i, len(df)):
            if df.loc[j, 'holiday_type']=='RH':
                df.loc[i, 'next_RH'] = j-i
                break
    all_dfs_tmp.append(df)
all_dfs = all_dfs_tmp

# Merge and download final DF to CSV

In [ ]:
df_final_holidays = pd.DataFrame()
df_final_list = []
for df in all_dfs:
    frames = [df_final_holidays, df]
    df_final_holidays = pd.concat(frames)
df_final_holidays

,Date,weekdays,OUTFLOW,INFLOW,BRANCH_ID,is_holiday,holiday_name,holiday_type,holiday_sequence,RH_in_7,RH_in_15,RH_in_30,next_RH
0,2018-08-02,Thursday,0,0,1,0,,,WHH,0,0,1,19
1,2018-08-03,Friday,0,0,1,1,Weekly,WK,HHW,0,0,1,18
2,2018-08-04,Saturday,0,0,1,1,Weekly,WK,HWW,0,0,1,17
3,2018-08-05,Sunday,0,0,1,0,,,WWW,0,0,1,16
4,2018-08-06,Monday,0,0,1,0,,,WWW,0,1,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,2022-07-15,Friday,0,0,4,1,Weekly,WK,HHW,0,0,0,200
1444,2022-07-16,Saturday,0,0,4,1,Weekly,WK,HWW,0,0,0,200
1445,2022-07-17,Sunday,0,0,4,0,,,WWW,0,0,0,200
1446,2022-07-18,Monday,0,0,4,0,,,WWW,0,0,0,200


In [ ]:
df_final_holidays.to_csv('transDataHolidays.csv', encoding = 'utf-8-sig') 
files.download('transDataHolidays.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>